# Проект по SQL

**Цель исследования**

Проанализировать базу данных, в которой хранится информация о книгах, издательствах, авторах, а также пользовательские
обзоры книг и ответить на вопросы:

* Посчитайте, сколько книг вышло после 1 января 2000 года;
* Для каждой книги посчитайте количество обзоров и среднюю оценку;
* Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
* Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
* Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Описание данных

`Таблица books`

Содержит данные о книгах:

* book_id — идентификатор книги;
* author_id — идентификатор автора;
* title — название книги;
* num_pages — количество страниц;
* publication_date — дата публикации книги;
* publisher_id — идентификатор издателя.

`Таблица authors`

Содержит данные об авторах:

* author_id — идентификатор автора;
* author — имя автора.

`Таблица publishers`

Содержит данные об издательствах:

* publisher_id — идентификатор издательства;
* publisher — название издательства;

`Таблица ratings`

Содержит данные о пользовательских оценках книг:

* rating_id — идентификатор оценки;
* book_id — идентификатор книги;
* username — имя пользователя, оставившего оценку;
* rating — оценка книги.

`Таблица reviews`

Содержит данные о пользовательских обзорах:

* review_id — идентификатор обзора;
* book_id — идентификатор книги;
* username — имя автора обзора;
* text — текст обзора.

## ER диаграмма

![title](data/ER_diagram.png)

## Подключение к базе данных

Импортируем библиотеки

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
db_config['pwd'],
db_config['host'],
db_config['port'],
db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

Создадим простую функцию, которая нам будет возвращать результат запроса.

In [3]:
def select(query):
    return pd.io.sql.read_sql(query, con = engine)

## Посчитаем, сколько книг вышло после 1 января 2000 года

In [4]:
query = """ 
SELECT COUNT(*)
FROM books
WHERE publication_date > '2000-01-01'
"""

In [5]:
select(query)

,count
0,819


После 1 января 2000 года вышло 819 книги.

## Для каждой книги посчитаем количество обзоров и среднюю оценку

Так как в ТЗ нас просят посчитать **для каждой книги**, а в таблице обзоров обзоры есть не на каждую книгу, вычислим 0 для тех книг, где обзоров нет.

In [6]:
query = """
SELECT b.book_id,
	b.title AS book_title,
	AVG(r.rating) AS average_rating,
	COUNT(DISTINCT CASE 
			WHEN re.review_id IS NOT NULL
				THEN re.review_id
			END) AS reviews_count
FROM books AS b
LEFT JOIN reviews AS re ON b.book_id = re.book_id
LEFT JOIN ratings AS r ON b.book_id = r.book_id
GROUP BY b.book_id,
	b.title
ORDER BY 1
"""

In [7]:
select(query)

,book_id,book_title,average_rating,reviews_count
0,1,'Salem's Lot,3.666667,2
1,2,1 000 Places to See Before You Die,2.500000,1
2,3,13 Little Blue Envelopes (Little Blue Envelope...,4.666667,3
3,4,1491: New Revelations of the Americas Before C...,4.500000,2
4,5,1776,4.000000,4
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),3.666667,3
996,997,Xenocide (Ender's Saga #3),3.400000,3
997,998,Year of Wonders,3.200000,4
998,999,You Suck (A Love Story #2),4.500000,2


## Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [8]:
query = """
SELECT publisher
FROM books AS b
JOIN publishers AS p ON b.publisher_id = p.publisher_id
WHERE num_pages > 50
GROUP BY publisher
ORDER BY COUNT(*) DESC LIMIT 1
"""

In [9]:
select(query)

,publisher
0,Penguin Books


Издательство Penguin Books выпустило наибольшее число книг толще 50 страниц.

## Определим автора с самой высокой средней оценкой книг — учитываем только книги с 50 и более оценками.

In [10]:
query = """
SELECT a.author
FROM books AS b
LEFT JOIN ratings AS r ON b.book_id = r.book_id
LEFT JOIN authors AS a ON b.author_id = a.author_id
GROUP BY b.title,
	a.author_id,
	r.book_id
HAVING COUNT(username) > 50
ORDER BY avg(rating) DESC LIMIT 1
"""

In [11]:
select(query)

,author
0,J.K. Rowling/Mary GrandPré


J.K. Rowling/Mary GrandPré - автор с самой высокой средней оценкой книг среди книг с 50 и более оценками.

## Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [12]:
query = """
SELECT AVG(t.count_of_reviews)
FROM (
	SELECT r.username,
		COUNT(book_id) AS count_of_reviews
	FROM reviews AS r
	JOIN (
		SELECT username
		FROM ratings
		GROUP BY username
		HAVING COUNT(book_id) > 50
		) ra ON ra.username = r.username
	GROUP BY r.username
	) AS t
""" 

In [13]:
select(query)

,avg
0,24.333333


Среднее количество обзоров от пользователей, которые поставили больше 50 оценок равно 24.3.